# 13 - Claw Contraption

https://adventofcode.com/2024/day/13


In [87]:
// Imports

import colors from "../../utils/colors.ts"
import objects from "../../utils/objects.ts"
import strings from "../../utils/strings.ts"
import numbers from "../../utils/numbers.ts"
import plots from "../../utils/plots.ts";

In [88]:
// Read Input

const file = await Deno.readTextFile("input-base.txt");
class CoOrds {
    x: number;
    y: number;

    constructor(x: string, y: string) {
        this.x = parseInt(x);
        this.y = parseInt(y);
    }

    equals(other: CoOrds): boolean {
        return this.x === other.x && this.y === other.y;
    }
}

class ClawMachine {
    prize: CoOrds;
    a: CoOrds;
    b: CoOrds;

    constructor(prize: CoOrds, a: CoOrds, b: CoOrds) {
        this.prize = prize;
        this.a = a;
        this.b = b;
    }
}

const machines = [];
const lines = file.split("\n");
for (let i = 0; i < lines.length; i += 4) {
    const [, ax, ay] = lines[i].match(/Button A: X\+(\d+), Y\+(\d+)/);
    const [, bx, by] = lines[i + 1].match(/Button B: X\+(\d+), Y\+(\d+)/);
    const [, px, py] = lines[i + 2].match(/Prize: X=(\d+), Y=(\d+)/);
    machines.push(new ClawMachine(new CoOrds(px, py), new CoOrds(ax, ay), new CoOrds(bx, by)));
}
machines

[
  ClawMachine {
    prize: CoOrds { x: 8400, y: 5400 },
    a: CoOrds { x: 94, y: 34 },
    b: CoOrds { x: 22, y: 67 }
  },
  ClawMachine {
    prize: CoOrds { x: 12748, y: 12176 },
    a: CoOrds { x: 26, y: 66 },
    b: CoOrds { x: 67, y: 21 }
  },
  ClawMachine {
    prize: CoOrds { x: 7870, y: 6450 },
    a: CoOrds { x: 17, y: 86 },
    b: CoOrds { x: 84, y: 37 }
  },
  ClawMachine {
    prize: CoOrds { x: 18641, y: 10279 },
    a: CoOrds { x: 69, y: 23 },
    b: CoOrds { x: 27, y: 71 }
  }
]

In [89]:
// Part 1 - What is the fewest tokens you would have to spend to win all possible prizes? 

const MAX_ATTEMPTS = 100; // Brute-force is fine for this small number of attempts
const calculateCost = (a, b) => 3 * a + b;

const findMinCostToWinPrize = (machine) => {
    const maxStepsForA = Math.min(Math.ceil(machine.prize.x / machine.a.x), MAX_ATTEMPTS);
    const maxStepsForB = Math.min(Math.ceil(machine.prize.x / machine.b.x), MAX_ATTEMPTS);
    let minCost = Infinity;
    for (let i = 0; i <= maxStepsForA; i++) {
        for (let j = 0; j <= maxStepsForB; j++) {
            const x = i * machine.a.x + j * machine.b.x;
            const y = i * machine.a.y + j * machine.b.y;
            if (machine.prize.equals(new CoOrds(x, y))) {
                const cost = calculateCost(i, j);
                if (cost < minCost) {
                    minCost = cost;
                }
            }
        }
    }
    return minCost
}
machines.reduce((acc, machine) => {
    const cost = findMinCostToWinPrize(machine);
    if (!Number.isFinite(cost)) return acc;
    return {
        wins: acc.wins + 1,
        cost: acc.cost + cost
    }

}, { wins: 0, cost: 0 })

{ wins: 2, cost: 480 }

In [90]:
// Part 2 - What is the fewest tokens you would have to spend to win all possible prizes with no attempts limit ?

const THRESHOLD_ADD = 10000000000000;

const findMinCostToWinPrizeV2 = (machine) => {
    // ax + by = c, There is only one intersection or none for two lines
    const [a1, a2] = [machine.a.x, machine.a.y];
    const [b1, b2] = [machine.b.x, machine.b.y];
    const [c1, c2] = [machine.prize.x, machine.prize.y];

    const denominator = a1 * b2 - a2 * b1;
    if (denominator === 0) {
        // No intersection
        return Infinity;
    }
    const x = (c1 * b2 - c2 * b1) / denominator;
    const y = (a1 * c2 - a2 * c1) / denominator;

    if (x > 0 && y > 0 && Number.isInteger(x) && Number.isInteger(y)) {
        return calculateCost(x, y);
    }
    return Infinity;
};

machines.reduce((acc, machine) => {
    const modifiedMachine = objects.deepCopy(machine);
    modifiedMachine.prize.x += THRESHOLD_ADD;
    modifiedMachine.prize.y += THRESHOLD_ADD;
    const cost = findMinCostToWinPrizeV2(modifiedMachine);
    if (!Number.isFinite(cost)) return acc;
    return {
        wins: acc.wins + 1,
        cost: acc.cost + cost
    }

}, { wins: 0, cost: 0 });

{ wins: 2, cost: 875318608908 }